In [1]:
%pip install -q bqplot ipyleaflet ipywidgets==8.0

In [2]:
import os
import urllib
import json
from datetime import datetime

import numpy as np
import pandas as pd
import geopandas as gpd

from js import fetch

from ipywidgets import Dropdown, Output

from bqplot import Lines, Figure, LinearScale, DateScale, Axis

from ipyleaflet import Map, GeoJSON, WidgetControl, GeoData, LayersControl

## Fetch the data

In [3]:
URL = "https://firenrt.delta-backend.com/collections/public.eis_fire_snapshot_perimeter_nrt"
res = await fetch(URL)
text = await res.text()

most_recent_time = json.loads(text)["extent"]["temporal"]["interval"][0][1]
most_recent_time

'2023-07-25T12:00:00+00:00'

In [4]:
async def get_data(url):
    response = await fetch(url)
    text = await response.text()
    result = json.loads(text)
    assert result["numberMatched"] == result["numberReturned"], "Increase the limit"
    return gpd.GeoDataFrame.from_features(result["features"])

In [5]:
url = (
    'https://firenrt.delta-backend.com/'
    'collections/public.eis_fire_snapshot_perimeter_nrt/items?'
    f'datetime={urllib.parse.quote(most_recent_time)}&'
    'limit=1000&'
    'filter=farea%3E5+AND+duration%3E2&'
    'f=geojson'
)

df = await get_data(url)
print("Found", len(df), "fires burning at most recent time:", most_recent_time)

Found 16 fires burning at most recent time: 2023-07-25T12:00:00+00:00


In [6]:
fireids = ",".join(str(fireid) for fireid in df.fireid)
url = (
    "https://firenrt.delta-backend.com/collections/public.eis_fire_snapshot_fireline_nrt/items?"
    f"filter=fireid+IN+({fireids})&"
    "limit=1000"
)
flines = await get_data(url)

In [7]:
def clean_data(data):
    data.fireid = data.fireid.astype(str)
    data.fireid = data.fireid.apply(lambda x: f'F{x}' if not x.startswith("F") else x)
    # data.t = pd.to_datetime(data.t)
    data = data.sort_values("t", ascending=False)
    return data

df = clean_data(df)
flines = clean_data(flines)
pins = df.copy()
pins["geometry"] = df.geometry.representative_point()

In [8]:
fireids = ",".join(f"'{fireid}'" for fireid in df.fireid)
url = (
    "https://firenrt.delta-backend.com/collections/public.eis_fire_lf_perimeter_nrt/items?"
    f"filter=fireid+IN+({fireids})&"
    "limit=1000"
)
fires_df = await get_data(url)
fires_df = clean_data(fires_df)

print("Found", len(fires_df), "time steps for current fires")

Found 351 time steps for current fires


## Construct a timeseries

In [9]:
fireid = df.fireid[0]
data_name = 'farea'

data = fires_df
subset = data[data.fireid == fireid].sort_values("t")

y_data = subset[data_name].values
x_data = pd.to_datetime(subset["t"].values)
t = x_data.max()

In [10]:
y_scale = LinearScale()
date_scale = DateScale()

lines = Lines(x=x_data, y=y_data, scales={'x': date_scale, 'y': y_scale})
v_lines = Lines(x=[t, t], y=[y_data.min(), y_data.max()], scales={'x': date_scale, 'y': y_scale}, colors=["red"])

ax_x = Axis(label='Time', scale=date_scale)
ax_y = Axis(label=data_name.capitalize(), scale=y_scale, orientation='vertical', side='left')

figure = Figure(axes=[ax_x, ax_y], title=fireid, marks=[lines, v_lines], animation_duration=500,
                layout={'max_height': '400px', 'width': '500px'})

In [11]:
figure

Figure(animation_duration=500, axes=[Axis(label='Time', scale=DateScale()), Axis(label='Farea', orientation='v…

In [12]:
def update_figure(fireid, data_name, t):
    try:
        subset = data[data.fireid == fireid].sort_values("t")
        y_data = subset[data_name].values
        x_data = pd.to_datetime(subset["t"].values)
        lines.y = y_data
        lines.x = x_data
        v_lines.x = [t, t]
        v_lines.y = [lines.y.min(), lines.y.max()]
        ax_y.label = data_name.capitalize()
        figure.title = fireid
    except IndexError:
        pass
    
def update_y(fireid, data_name):
    try:
        subset = data[data.fireid == fireid].sort_values("t")
        lines.y = subset[data_name].values
        v_lines.y = [lines.y.min(), lines.y.max()]
        ax_y.label = data_name.capitalize()

    except IndexError:
        pass

In [13]:
update_figure(df.fireid.sample().item(), "farea", t)

## Construct a map

In [19]:
m = Map(center=(39, -98), zoom=4, scroll_wheel_zoom=True)

m.layout.height="600px"

current_points = GeoData(geo_dataframe=pins, name = 'Pins')

current_polygons = GeoData(
    geo_dataframe=df, 
    style={'fillColor': 'red','color': 'red'}, 
    hover_style={'fillColor': 'black'}, 
    name='Current Perimeters'
)

archive_polygons = GeoData(
    geo_dataframe=fires_df, 
    style={'color': 'black', "fillOpacity": 0}, 
    hover_style={'fillColor': 'black', "fillOpacity": 0.5}, 
    name='Archive Perimeters'
)

fline_paths = GeoData(
    geo_dataframe=flines,
    style={'color': 'orange'},
    name='Fire Lines'
)

out = Output()

dropdown = Dropdown(
    options=['farea', 'n_pixels', 'flinelen'],
    value=data_name,
    description='Plotting:'
)

m.add(current_points)
m.add(current_polygons)
m.add(archive_polygons)
m.add(fline_paths)
m.add(LayersControl())
m.add(WidgetControl(widget=out, position="bottomleft"))
m.add(WidgetControl(widget=figure, position='bottomright'))
m.add(WidgetControl(widget=dropdown, position='topright'))

def display_properties(feature, **kwargs):
    out.clear_output()
    with out:
        display(pd.Series(feature["properties"]))
        
def on_hover(event, feature, **kwargs):
    global fireid
    global t
    
    fireid = feature['properties']['fireid']
    t = pd.to_datetime(feature['properties']['t'])

    update_figure(fireid, data_name, t)
    display_properties(feature)
    
    
def on_dropdown(change):
    global data_name

    data_name = change['new']
    update_y(fireid, data_name)

dropdown.observe(on_dropdown, 'value')
current_polygons.on_hover(on_hover)
archive_polygons.on_hover(on_hover)
current_points.on_click(on_hover)

m

Map(center=[39, -98], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…